In [9]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import os
import matplotlib.pyplot as plt
import sklearn
import numpy as np

In [13]:
tf.config.experimental.list_logical_devices('GPU')

[LogicalDevice(name='/device:GPU:0', device_type='GPU')]

<p>idea is to create a vector of number of terms and expecting level of toxicity based on number of terms TRUE<br>USing RNN to create tokenized datasets</p>

In [7]:
csv_path=os.path.join('data','train','train.csv')
df=pd.read_csv(csv_path)
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [12]:
df.iloc[6]['comment_text']

'COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK'

In [84]:
col_names=df.columns[2:]
col_names

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [14]:
df[df.columns[2:]]

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
159566,0,0,0,0,0,0
159567,0,0,0,0,0,0
159568,0,0,0,0,0,0
159569,0,0,0,0,0,0


In [15]:
df[df.columns[2:]].values

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [18]:
x=df['comment_text']#df
y=df[df.columns[2:]].values#array
x

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

In [19]:
MAX_FEATURES=200000

In [20]:
vectorizer=tf.keras.layers.TextVectorization(max_tokens=MAX_FEATURES,
    output_sequence_length=1800,
    output_mode='int'
)

In [ ]:
vectorizer.adapt(x.values)#creating a vocabulary of words mapped to int

In [25]:
vectorizer('hello suckers dip at life')[:5]

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([  286, 24031, 17987,    34,   305], dtype=int64)>

In [26]:
vectorized_text=vectorizer(x.values)

In [27]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  643,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2506, ...,     0,     0,     0],
       [  425,   440,    70, ...,     0,     0,     0],
       ...,
       [32141,  7329,   383, ...,     0,     0,     0],
       [    5,    12,   533, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [ ]:
tf.data.Dataset.from_tensor_slices??#It splits the input tensor(s) along the first dimension (axis 0) into individual elements. Each slice becomes a single element in the resulting dataset.

Signature: tf.data.Dataset.from_tensor_slices(tensors, name=None)
Source:   
  @staticmethod
  def from_tensor_slices(tensors, name=None):
    """Creates a `Dataset` whose elements are slices of the given tensors.

    The given tensors are sliced along their first dimension. This operation
    preserves the structure of the input tensors, removing the first dimension
    of each tensor and using it as the dataset dimension. All input tensors
    must have the same size in their first dimensions.

    >>> # Slicing a 1D tensor produces scalar tensor elements.
    >>> dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
    >>> list(dataset.as_numpy_iterator())
    [1, 2, 3]

    >>> # Slicing a 2D tensor produces 1D tensor elements.
    >>> dataset = tf.data.Dataset.from_tensor_slices([[1, 2], [3, 4]])
    >>> list(dataset.as_numpy_iterator())
    [array([1, 2], dtype=int32), array([3, 4], dtype=int32)]

    >>> # Slicing a tuple of 1D tensors produces tuple elements containing
    

In [29]:
dataset=tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset=dataset.cache().shuffle(160000).batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)

In [45]:
len(dataset)
dataset.as_numpy_iterator().next()

(array([[ 877, 9356,  205, ...,    0,    0,    0],
        [   8,   72,    7, ...,    0,    0,    0],
        [   8,   47,   52, ...,    0,    0,    0],
        ...,
        [ 794,   10,   25, ...,    0,    0,    0],
        [ 717,  283, 1066, ...,    0,    0,    0],
        [  49,  234,   32, ...,    0,    0,    0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

In [46]:
batch_x,batch_y=dataset.as_numpy_iterator().next()

In [47]:
train=dataset.take(int(len(dataset)*.7))
test=dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
val=dataset.skip(int(len(dataset)*.7)).skip(int(len(dataset)*.2))

In [50]:
len(val)

999

<h2>Sequential Model with embedding layer</h2>

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Bidirectional, LSTM, Embedding, Flatten

In [55]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [ ]:
model=Sequential([
    Embedding(MAX_FEATURES+1,32),#inout dims, output dims
    Bidirectional(LSTM(32,activation='tanh')),
    Dense(128,activation='relu'),
    Dense(256,activation='relu'),
    Dense(128,activation='relu'),
    Dense(6,activation='sigmoid'),
])

In [60]:
model.compile(
    optimizer='adam'
    ,loss=tf.keras.losses.BinaryCrossentropy()
    ,metrics=['accuracy']
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [61]:
history=model.fit(train,epochs=1,validation_data=val)

6981/6981 [==============================] - 1007s 143ms/step - loss: 0.0623 - accuracy: 0.9823 - val_loss: 0.0478 - val_accuracy: 0.9937


In [62]:
history.history

{'loss': [0.06229091063141823],
 'accuracy': [0.9822912216186523],
 'val_loss': [0.047821491956710815],
 'val_accuracy': [0.9936760663986206]}

In [69]:
inp_text=vectorizer('you freaking suck')
res=model.predict(np.expand_dims(inp_text,0))
res[0]

1/1 [==============================] - 0s 154ms/step


array([0.9795185 , 0.18015602, 0.89957935, 0.04628459, 0.7557972 ,
       0.1671045 ], dtype=float32)

In [88]:
col_names

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [85]:
def predict(text):
    inp_text=vectorizer(text)
    res=model.predict(np.expand_dims(inp_text,0))
    for i in range(6):
        if(res[0][i]>0.5):
            print(col_names[i])

In [89]:
predict('you freaking suck i will kill you')

1/1 [==============================] - 0s 261ms/step
toxic
obscene
insult


In [90]:
predict('thank you for your service')

1/1 [==============================] - 0s 54ms/step


In [99]:
batch_x,batch_y=test.as_numpy_iterator().next()

In [97]:
model.predict(batch_x)>0.5

1/1 [==============================] - 0s 303ms/step


array([[False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [ True, False,  True, False,  True, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False]])

In [100]:
model.evaluate(batch_x,batch_y)

1/1 [==============================] - 0s 235ms/step - loss: 0.0014 - accuracy: 1.0000


[0.0014332156861200929, 1.0]

In [101]:
from tensorflow.keras.metrics import Precision, Recall,CategoricalAccuracy
pre=Precision()
re=Recall()
acc=CategoricalAccuracy()

In [102]:
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    X_true, y_true = batch
    # Make a prediction 
    yhat = model.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 89ms/step


In [103]:

print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.8983169794082642, Recall:0.5482168197631836, Accuracy:0.4729187488555908


In [104]:
import gradio as gr

In [105]:
model.save('toxicModel.h5')

In [106]:
model=tf.keras.models.load_model('toxicModel.h5')


In [108]:
import gradio as gr

def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx] > 0.5)
    
    return text

# Use `gr.Textbox` and `gr.Text` instead of `gr.inputs.Textbox` and `outputs='text'`
interface = gr.Interface(
    fn=score_comment,
    inputs=gr.Textbox(lines=2, placeholder='Comment to score'),
    outputs=gr.Text()
)

interface.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [110]:
interface.launch(share=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----


OSError: [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software: 'c:\\Users\\ujwal\\anaconda3\\envs\\py310\\lib\\site-packages\\gradio\\frpc_windows_amd64_v0.3'